In [1]:
import json
import re
import pandas as pd

In [2]:
with open("triggerData2016_runInfo.json", 'r') as f:
    runinfo = json.load(f)

with open("triggerData2016_hltprescales.json", 'r') as f:
    hlt_prescales = json.load(f)
    
with open("triggerData2016_l1prescales.json", 'r') as f:
    l1_prescales = json.load(f)

In [3]:
import urllib.request
with urllib.request.urlopen("http://www.hep.ph.ic.ac.uk/~sdb15/Analysis/ZinvWidth/data/json/Cert_271036-284044_13TeV_23Sep2016ReReco_Collisions16_JSON.txt") as f:
    cert_lumi = json.load(f)

In [4]:
def list_paths(path_regex):
    opaths = []
    for run in sorted(cert_lumi.keys()):
        rinf = runinfo[run]
        hltkey = rinf["hlt_menu"]
                
        paths = [col[1].split("_v")[0] for col in hlt_prescales[hltkey][1:]]
        
        opaths.extend([p for p in paths if path_regex.search(p)])
    return sorted(list(set(opaths)))

In [11]:
list_paths(re.compile("HLT_.*MET.*IsoTrk.*"))

['HLT_MET60_IsoTrk35_Loose', 'HLT_MET75_IsoTrk50', 'HLT_MET90_IsoTrk50']

In [6]:
def merge_lumi_lists(list1, list2):
    if len(list1)==0:
        return list2
    if len(list2)==0:
        return list1
    
    merged = []
    idx1, idx2 = 0, 0
    while idx1<len(list1) and idx2<len(list2):
        cand1 = list1[idx1]
        cand2 = list2[idx2]
        
        if cand1[0] < cand2[0]:
            if len(merged)>0 and cand1[0] <= merged[-1][1]+1:
                merged[-1][1] = cand1[1]
                idx1 += 1
                continue
            mini, maxi = cand1[0], cand1[1]
            idx1 += 1
        else:
            if len(merged)>0 and cand2[0] <= merged[-1][1]+1:
                merged[-1][1] = cand2[1]
                idx2 += 1
                continue
            mini, maxi = cand2[0], cand2[1]
            idx2 += 1
            if cand1[0] <= cand2[1]+1:
                maxi = cand1[1]
                idx1 += 1
        merged.append([mini, maxi])
    while idx1<len(list1):
        cand1 = list1[idx1]
        if len(merged)>0 and cand1[0] <= merged[-1][1]+1:
            merged[-1][1] = cand1[1]
        else:
            merged.append(cand1)
        idx1 += 1
    while idx2<len(list2):
        cand2 = list2[idx2]
        if len(merged)>0 and cand1[0] <= merged[-1][1]+1:
            merged[-1][1] = cand2[1]
        else:
            merged.append(cand2)
        idx2 += 1
    return merged

In [7]:
merge_lumi_lists([[17, 31]], [[3, 16], [32, 1278]])

[[3, 1278]]

In [8]:
from collections import OrderedDict as odict

def check_prescales(trigger_paths):
    output = odict()
    l1_output = odict()
    for path in trigger_paths:
        prescaled_run_lumi = odict()
        l1_prescaled_run_lumi = odict()
        #print("{}".format(path))
        for run in sorted(cert_lumi.keys()):
            lumi_ranges = cert_lumi[run]
            rinf = runinfo[run]
            hltkey = rinf["hlt_menu"]
            l1key = rinf["trig_mode"]
            ps_cols = rinf["ps_cols"]
            
            paths = [col[1].split("_v")[0] for col in hlt_prescales[hltkey][1:]]
            l1seeds = [col[-1] for col in hlt_prescales[hltkey][1:]]
            indexes = [col[2:len(col)-1] for col in hlt_prescales[hltkey][1:]]
            
            if path not in paths:
                continue
                
            path_idx = paths.index(path)
            
            # Level-1
            l1paths = [col[1] for col in l1_prescales[l1key][1:]]
            l1indexes = [col[2:] for col in l1_prescales[l1key][1:]]
            l1_seed_paths = l1seeds[path_idx].split(" OR ")
            
            l1_lumi_lists = []
            l1_ps_val_lists = []
            
            for l1_seed_path in l1_seed_paths:
                l1_path_idx = l1paths.index(l1_seed_path.strip())
                
                l1_lumi_list = []
                l1_ps_val_list = []
                
                for col, lumis in ps_cols.items():
                    ps_val = l1indexes[l1_path_idx][int(col)]
                    l1_ps_val_list.append(int(ps_val))
                    if ps_val not in l1_ps_val_list:
                        l1_lumi_list.append(lumis)
                    else:
                        l1_lumi_list[-1] = merge_lumi_lists(l1_lumi_lists[-1], lumis)
                        
                l1_lumi_lists.append(l1_lumi_list)
                l1_ps_val_lists.append(l1_ps_val_list)
                
            l1_lumi_list = l1_lumi_lists[0]
            l1_ps_val_list = l1_ps_val_lists[0]
            for psval in l1_ps_val_lists:
                for idx in range(len(l1_ps_val_list)):
                    if psval[idx] < l1_ps_val_list[idx]:
                        l1_ps_val_list[idx] = psval[idx]
                        
            l1_ps_vals = {}
            for idx in range(len(l1_ps_val_list)):
                psval = l1_ps_val_list[idx]
                if psval not in l1_ps_vals:
                    l1_ps_vals[psval] = l1_lumi_list[idx]
                else:
                    l1_ps_vals[psval] = merge_lumi_lists(l1_ps_vals[psval], l1_lumi_list[idx])
            
            l1_merged_lumis = []
            for ps, lumis in l1_ps_vals.items():
                if ps>1 or ps<0:
                    l1_merged_lumis = merge_lumi_lists(l1_merged_lumis, lumis)
            if len(l1_merged_lumis)>0:
                l1_prescaled_run_lumi[str(run)] = l1_merged_lumis
                
            ps_vals = {}
            for col, lumis in ps_cols.items():
                ps_val = indexes[path_idx][int(col)]
                if ps_val not in ps_vals:
                    ps_vals[ps_val] = lumis
                else:
                    ps_vals[ps_val] = merge_lumi_lists(ps_vals[ps_val], lumis)
            
            merged_lumis = []
            for ps, lumis in ps_vals.items():
                if int(ps)!=1:
                #if int(ps)>1 or int(ps)<0:
                    merged_lumis = merge_lumi_lists(merged_lumis, lumis)
            if len(merged_lumis)>0:
                prescaled_run_lumi[str(run)] = merged_lumis
        output[path] = prescaled_run_lumi
        l1_output[path] = l1_prescaled_run_lumi
    return output, l1_output
                

In [13]:
hlt_prescaled, l1_prescaled = check_prescales(trigger_paths = [
#    "HLT_PFMETNoMu90_PFMHTNoMu90_IDTight",
#    "HLT_PFMETNoMu100_PFMHTNoMu100_IDTight",
#    "HLT_PFMETNoMu110_PFMHTNoMu110_IDTight",
#    "HLT_PFMETNoMu120_PFMHTNoMu120_IDTight",
#    "HLT_PFMET170_NotCleaned",
#    "HLT_PFMET170_BeamHaloCleaned",
#    "HLT_PFMET170_HBHECleaned",
#    "HLT_PFMET170_HBHE_BeamHaloCleaned",
    "HLT_MET75_IsoTrk50",
#    "HLT_IsoMu24",
#    "HLT_IsoTkMu24",
#    "HLT_TkMu50",
#    "HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL",
#    "HLT_Mu17_TrkIsoVVL_TkMu8_TrkIsoVVL",
#    "HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ",
#    "HLT_Mu17_TrkIsoVVL_TkMu8_TrkIsoVVL_DZ",
#    "HLT_Ele27_WPTight_Gsf",
#    "HLT_Ele105_CaloIdVT_GsfTrkIdT",
#    "HLT_Photon175",
#    "HLT_DoublePhoton60",
#    "HLT_PFJet450"
    
])
#print(l1_prescaled)
for k1, v1 in hlt_prescaled.items():
    print(k1)
    for k2, v2 in v1.items():
        print("\t{}: {}".format(k2, v2))

HLT_MET60_IsoTrk35_Loose
	273158: [[3, 1278]]
	273302: [[3, 481]]
	273402: [[3, 189]]
	273403: [[3, 63]]
	273404: [[3, 22]]
	273405: [[3, 28]]
	273406: [[3, 116]]
	273408: [[3, 7]]
	273409: [[3, 313]]
	273410: [[3, 97]]
	273411: [[3, 91]]
	273425: [[3, 60]]
	273446: [[3, 45]]
	273447: [[3, 419]]
	273448: [[3, 395]]
	273449: [[3, 215]]
	273450: [[3, 760]]
	273492: [[3, 54]]
	273493: [[3, 235]]
	273494: [[3, 201]]
	273502: [[3, 1071]]
	273503: [[3, 621]]
	273554: [[3, 441]]
	273555: [[3, 184]]
	273725: [[3, 2551]]
	273728: [[3, 103]]
	273730: [[3, 2831]]
	274094: [[3, 342]]
	274146: [[3, 73]]
	274157: [[3, 533]]
	274159: [[3, 42]]
	274160: [[3, 210]]
	274161: [[3, 535]]
	274172: [[3, 103]]
	274198: [[3, 191]]
	274199: [[3, 628]]
	274200: [[3, 692]]
	274240: [[3, 83]]
	274241: [[3, 1175]]
	274244: [[3, 610]]
	274250: [[3, 702]]
	274251: [[3, 560]]
	274283: [[3, 18]]
	274284: [[3, 210]]
	274286: [[3, 161]]
	274314: [[3, 164]]
	274315: [[3, 425]]
	274316: [[3, 969]]
	274317: [[3, 17]]
	2743